In [ ]:
from hashlib import sha256
import hashlib
import json
from uuid import uuid4
from time import time

In [ ]:
# ##Blueprint of Our Blockchain Class

# """ 
# Let's Create a Class name BlockChain which create initial Empty List for storing the Blockchian and transcation
# """

# class Blockchain(object):
    
#     def __init__(self):
#         """
#         Create two empty list one for Blockchain, and another for Transcation
#         """
        
#         self.chain = []
#         self.current_transcations = []
        
#     def new_block(self):
#         """ Create a new Block and adds it to the chain """
#         pass
        
#     def new_transcation(self):
#         """ Add a new Transcation to the transcations list """
#         pass
    
#     @staticmethod
#     def hash(block):
#         """ return hash of the given Block """
        
#     @property
#     def last_block(self):
#         """ Pythonic way to return the last time of the list """
#         pass

In [ ]:

class Blockchain(object):
    
    def __init__(self):
        """
        Create two empty list one for Blockchain, and another for Transcation
        """
        
        self.chain = []
        self.current_transcations = []
        
        # create the genesis block
        self.new_block(previous_hash=1, proof=100)
        
    def new_block(self, previous_hash, proof):
        """ 
        Create a new Block and adds it to the Blockchain 
        :param previous_hash: <str> Hash of the previous Block
        :param proof: <int> The proof given by the proof of work algorithm
        :return: <dict> New Block
        """
        
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(),
            'transactions' : self.current_transcations,
            'proof' : proof,
            'previous_hash' : previous_hash
        }
    
        # Reset the current list of the transcation
        self.current_transcations = []
        self.chain.append(block)
        return block
        
    def new_transcation(self, sender, recipient, amount):
        """
        Create a new Transcation to go into the next mind Block
        :param sender: <str> Address to the Sender
        :param recipient: <str> Address to the Recipient
        :param amount: <int> Amount
        :return: <int> The index of the block that hold the transcation
        """
        
        self.current_transcations.append({
            'sender' : sender,
            'recipient' : recipient,
            'amount' : amount,
        })
    
        return self.last_block['index']+1
    @staticmethod
    def hash(block):
        """ 
        Create a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """
        
        # we must make sure that dict is Orderd, else we'll have Inconsistent Hash
        block_string = json.dumps(block, sort_keys=True).encode()
        return sha256(block_string).hexdigest()
        
        
    @property
    def last_block(self):
        """ Pythonic way to return the last time of the list """
        self.chain[-1]
        
    def proof_of_work(self, last_proof):
        """ 
        simple Proof of Work's Algorithm
        - Find a number p' such that hash(pp') contain leading 4 zeros, where p is the previous p'
        - p is the previous proof, p' is the new proof
        :param last_proof: <int>
        :return: <int>
        """
        
        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
        return proof
    
    @staticmethod
    def valid_proof(last_proof, proof):
        """
        Validate the proof: Does hash(last_proof, proof) contatin 4 leading zeroes?
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :return: <bool> True if correct, False incase of Incorrect
        """
        
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

In [ ]:

# block = {
#             'index' : 1,
#             'timestamp' : time(),
#             'transactions' : {
#                 'sender' : "8527147fe1f5426f9dd545de4b27ee00",
#                 'recipient' : "a77f5cdfa2934df3954a5c7c7da5df1f",
#                 'amount' : 5,
#             },
#             'proof' : 324984774000,
#             'previous_hash' : "2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824"
#         }

In [ ]:
from flask import Flask
from textwrap import dedent

In [ ]:
# Intantiate the node
app = Flask(__name__)

In [ ]:
# Genrate a globally unique address for this node
node_identifier = str(uuid4()).replace('-','')

In [ ]:
blockchain = Blockchain()


In [ ]:
@app.route('/mine', methods=['GET'])
def mine():
    # we run the POW algorith to get the next proof
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)
    
    # we must reward for finding the proof
    # The sender is "0" to signify that this node has mined a new coin
    
    blockchain.new_transcation(
        sender = "0",
        recipient = node_identifier,
        amount = 1,
    )
    
    # Forge the new block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    
    response = {
        'Message' : "New block Forged",
        'index' : block['index'],
        'transactions' : block['transactions'],
        'proof' : block['proof'],
        'previous_hash' : block['previous_hash'],
    }
    
    return jsonify(response), 200
    
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()
    #check that the requires field in the post data
    requires = ['sender', 'recipient', 'amount']
    if not all(k in values for k in requires ):
        return "Missing Values", 400
    
    #create a new transaction
    index = blockchain.new_transcation(values['sender'],values['recipient'],values['amount'])
    
    response = {'message' : f'Transaction will be added to Block{index}'}
    return jsonify(response), 201
@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain' : blockchain.chain,
        'length' : len(blockchain.chain)
    }
    return jsonify(response),200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)